In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import re
import random
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
from tqdm import tqdm
from glob import glob
import tensorflow as tf 
warnings.filterwarnings(action='ignore')


In [2]:
path = 'D:/python_project/hyunmin_project/운동동작분류/'

train=pd.read_csv(path + 'open/train_features.csv')
train_label=pd.read_csv(path + 'open/train_labels.csv')
test=pd.read_csv(path + 'open/test_features.csv')
submission=pd.read_csv(path + 'open/sample_submission.csv')

In [3]:
train.shape, train_label.shape, test.shape, submission.shape

((1875000, 8), (3125, 3), (469200, 8), (782, 62))

In [4]:
X = tf.reshape(np.array(train.iloc[:,2:]),[-1, 600, 6])
X = np.asarray(X)

In [5]:
y = train_label['label'].values

In [6]:
labels = np.arange(0,3125)

In [7]:
X.shape, y.shape, labels.shape

((3125, 600, 6), (3125,), (3125,))

In [8]:
X_augmented = [] 
y_augmented = []  
l_augmented = [] 
idx = 3125
for i in tqdm(range(X.shape[0]), position = 0, leave = True): 
    for j in range(10): 
        shifted = np.roll(X[i], int(random.random() * 600)) 
        X_augmented.append(shifted) 
        y_augmented.append(y[i])     
        l_augmented.append(idx) 
        idx += 1 
X_augmented = np.asarray(X_augmented) 
y_augmented = np.asarray(y_augmented)
l_augmented = np.asarray(l_augmented) 
X_train = np.concatenate([X, X_augmented]) 
Y_train = np.concatenate([y, y_augmented])
l_train = np.concatenate([labels, l_augmented])  


100%|█████████████████████████████████████████████████████████████████████████| 3125/3125 [00:01<00:00, 3021.56it/s]


In [9]:
X_train.shape

(34375, 600, 6)

In [10]:
ld_train_add = [] 
for i in tqdm(range(len(l_train))):
    for j in range(600): 
        ld_train_add.append(l_train[i]) 
ld_train_add = np.asarray(ld_train_add)

100%|███████████████████████████████████████████████████████████████████████| 34375/34375 [00:04<00:00, 8200.50it/s]


In [11]:
train_df = pd.DataFrame(np.concatenate(X_train)) 
train_df.insert(0,'id',ld_train_add) 
train_df.head()

,id,0,1,2,3,4,5
0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234


In [12]:
features = ['id',0,1,2,3,4,5]
train_df_processed = train_df[features].groupby('id').agg(['max', 'min', 'mean']) 

In [13]:
train_df_processed.shape

(34375, 18)

In [14]:
colum_list = []
for i in range(int(train_df_processed.shape[1])):
    colum_list.append(i)

In [15]:
train_df_processed.columns = colum_list
train_df_processed

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
id,,,,,,,,,,,,,,,,,,
0,1.344268,0.591940,0.931329,0.176871,-0.624113,-0.218471,-0.054876,-0.786336,-0.370422,31.644123,-46.254836,-1.865269,69.847244,-85.887677,-3.359506,55.953827,-79.930029,1.182107
1,1.234020,-2.156208,-0.766580,0.700065,-1.295598,-0.317258,0.888661,-1.019531,-0.004223,286.624363,-325.328531,11.071600,389.608060,-315.096003,1.740475,340.170199,-270.980823,1.393294
2,1.219836,-1.142847,0.039836,0.650645,-0.690990,-0.082403,1.332992,0.073846,0.626012,73.525082,-164.779067,-8.472951,297.320834,-249.953944,0.597877,55.642836,-44.192071,3.053291
3,-0.622250,-1.417751,-0.887702,0.283721,-0.540827,-0.087668,0.598814,-0.257124,0.227357,192.765368,-69.419166,17.744167,159.083788,-82.537304,4.800931,56.456908,-85.600536,-5.869898
4,0.599720,-2.429109,-0.659018,1.724782,-2.055076,-0.337067,2.678034,-1.250483,0.202758,613.972600,-769.076518,-4.819638,284.952954,-243.909948,9.651713,221.015193,-270.581913,4.453382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34370,-0.193247,-0.952761,-0.623010,0.543977,-0.380657,0.226848,52.957569,-58.732727,-1.638705,90.173541,-134.268304,-1.226870,80.302840,-100.845205,-3.246825,0.538809,-1.013813,-0.434048
34371,90.173541,-134.268304,-1.226870,80.302840,-100.845205,-3.246825,0.538809,-1.013813,-0.434048,-0.193247,-0.952761,-0.623010,0.543977,-0.380657,0.226848,52.957569,-58.732727,-1.638705
34372,90.173541,-134.268304,-1.226870,80.302840,-100.845205,-3.246825,0.538809,-1.013813,-0.434048,-0.193247,-0.952761,-0.623010,0.543977,-0.380657,0.226848,52.957569,-58.732727,-1.638705


In [51]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_df_processed, Y_train,
                                                    test_size=0.2, random_state=0)
x_train.shape

(27500, 18)

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV
import xgboost as xgb
import lightgbm as lgb
param_test1 = {
 'max_depth':[4,6,8,10],
 'min_child_weight':[1, 3, 5]
}
gsearch1 = GridSearchCV(estimator = lgb.LGBMClassifier(learning_rate=0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=5, 
                                                  min_child_weight=1, 
                                                  objective= 'multiclass', 
                                                  nthread=-1, 
                                                  eval_metric='logloss',
                                                  scale_pos_weight=1, seed=2019),
                        
param_grid = param_test1, scoring='neg_log_loss',n_jobs=-1,iid=False, cv=5, verbose=10)
gsearch1.fit(x_train, y_train, eval_set=[(x_test, y_test)], early_stopping_rounds=30, 
                  eval_metric="logloss")
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

In [53]:
X_train = train_df_processed

In [56]:
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
model=[]
Y_train = np.array(Y_train)
for train_idx, val_idx in k_fold.split(X_train):

    # split train, validation set
    X = X_train.iloc[train_idx]
    y = Y_train[train_idx]
    valid_x = X_train.iloc[val_idx]
    valid_y = Y_train[val_idx]

    lgb_model = lgb.LGBMClassifier(learning_rate=0.05, 
                                                  n_estimators=1000, 
                                                  max_depth=10, 
                                                  min_child_weight=5, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.6,
                                                  objective= 'multiclass', 
                                                  nthread=-1, 
                                                  eval_metric='logloss',
                                                  scale_pos_weight=1, seed=2019)
    
    lgb_model.fit(X, y, eval_set=[(valid_x, valid_y)], early_stopping_rounds=50, 
                  eval_metric="logloss")
    
    model.append(lgb_model)

[1]	valid_0's multi_logloss: 2.59185
Training until validation scores don't improve for 50 rounds
[2]	valid_0's multi_logloss: 2.46845
[3]	valid_0's multi_logloss: 2.36168
[4]	valid_0's multi_logloss: 2.26543
[5]	valid_0's multi_logloss: 2.17808
[6]	valid_0's multi_logloss: 2.09607
[7]	valid_0's multi_logloss: 2.01777
[8]	valid_0's multi_logloss: 1.94581
[9]	valid_0's multi_logloss: 1.88001
[10]	valid_0's multi_logloss: 1.81683
[11]	valid_0's multi_logloss: 1.75841
[12]	valid_0's multi_logloss: 1.70466
[13]	valid_0's multi_logloss: 1.65421
[14]	valid_0's multi_logloss: 1.60872
[15]	valid_0's multi_logloss: 1.5643
[16]	valid_0's multi_logloss: 1.52255
[17]	valid_0's multi_logloss: 1.48378
[18]	valid_0's multi_logloss: 1.44752
[19]	valid_0's multi_logloss: 1.41237
[20]	valid_0's multi_logloss: 1.381
[21]	valid_0's multi_logloss: 1.34977
[22]	valid_0's multi_logloss: 1.31994
[23]	valid_0's multi_logloss: 1.29233
[24]	valid_0's multi_logloss: 1.26608
[25]	valid_0's multi_logloss: 1.24066
[

[210]	valid_0's multi_logloss: 0.411654
[211]	valid_0's multi_logloss: 0.411062
[212]	valid_0's multi_logloss: 0.410523
[213]	valid_0's multi_logloss: 0.409869
[214]	valid_0's multi_logloss: 0.409314
[215]	valid_0's multi_logloss: 0.408752
[216]	valid_0's multi_logloss: 0.408166
[217]	valid_0's multi_logloss: 0.407627
[218]	valid_0's multi_logloss: 0.407081
[219]	valid_0's multi_logloss: 0.406529
[220]	valid_0's multi_logloss: 0.405997
[221]	valid_0's multi_logloss: 0.405479
[222]	valid_0's multi_logloss: 0.404939
[223]	valid_0's multi_logloss: 0.40428
[224]	valid_0's multi_logloss: 0.403794
[225]	valid_0's multi_logloss: 0.403261
[226]	valid_0's multi_logloss: 0.402743
[227]	valid_0's multi_logloss: 0.402295
[228]	valid_0's multi_logloss: 0.40188
[229]	valid_0's multi_logloss: 0.401432
[230]	valid_0's multi_logloss: 0.400853
[231]	valid_0's multi_logloss: 0.400412
[232]	valid_0's multi_logloss: 0.399975
[233]	valid_0's multi_logloss: 0.399529
[234]	valid_0's multi_logloss: 0.399065
[2

[418]	valid_0's multi_logloss: 0.356858
[419]	valid_0's multi_logloss: 0.356713
[420]	valid_0's multi_logloss: 0.356583
[421]	valid_0's multi_logloss: 0.356407
[422]	valid_0's multi_logloss: 0.356228
[423]	valid_0's multi_logloss: 0.356149
[424]	valid_0's multi_logloss: 0.356021
[425]	valid_0's multi_logloss: 0.355976
[426]	valid_0's multi_logloss: 0.355843
[427]	valid_0's multi_logloss: 0.355788
[428]	valid_0's multi_logloss: 0.355583
[429]	valid_0's multi_logloss: 0.355512
[430]	valid_0's multi_logloss: 0.355383
[431]	valid_0's multi_logloss: 0.355249
[432]	valid_0's multi_logloss: 0.355171
[433]	valid_0's multi_logloss: 0.355079
[434]	valid_0's multi_logloss: 0.354932
[435]	valid_0's multi_logloss: 0.354745
[436]	valid_0's multi_logloss: 0.354631
[437]	valid_0's multi_logloss: 0.354539
[438]	valid_0's multi_logloss: 0.354417
[439]	valid_0's multi_logloss: 0.354297
[440]	valid_0's multi_logloss: 0.354193
[441]	valid_0's multi_logloss: 0.354048
[442]	valid_0's multi_logloss: 0.353971


[629]	valid_0's multi_logloss: 0.34224
[630]	valid_0's multi_logloss: 0.342212
[631]	valid_0's multi_logloss: 0.342166
[632]	valid_0's multi_logloss: 0.342086
[633]	valid_0's multi_logloss: 0.342056
[634]	valid_0's multi_logloss: 0.34203
[635]	valid_0's multi_logloss: 0.341968
[636]	valid_0's multi_logloss: 0.341966
[637]	valid_0's multi_logloss: 0.341916
[638]	valid_0's multi_logloss: 0.341843
[639]	valid_0's multi_logloss: 0.34181
[640]	valid_0's multi_logloss: 0.341805
[641]	valid_0's multi_logloss: 0.341805
[642]	valid_0's multi_logloss: 0.341755
[643]	valid_0's multi_logloss: 0.341682
[644]	valid_0's multi_logloss: 0.341621
[645]	valid_0's multi_logloss: 0.341617
[646]	valid_0's multi_logloss: 0.341574
[647]	valid_0's multi_logloss: 0.341493
[648]	valid_0's multi_logloss: 0.341486
[649]	valid_0's multi_logloss: 0.341468
[650]	valid_0's multi_logloss: 0.341421
[651]	valid_0's multi_logloss: 0.341387
[652]	valid_0's multi_logloss: 0.341368
[653]	valid_0's multi_logloss: 0.341295
[65

[841]	valid_0's multi_logloss: 0.336559
[842]	valid_0's multi_logloss: 0.336557
[843]	valid_0's multi_logloss: 0.336573
[844]	valid_0's multi_logloss: 0.336605
[845]	valid_0's multi_logloss: 0.336584
[846]	valid_0's multi_logloss: 0.336546
[847]	valid_0's multi_logloss: 0.336566
[848]	valid_0's multi_logloss: 0.336606
[849]	valid_0's multi_logloss: 0.336592
[850]	valid_0's multi_logloss: 0.336579
[851]	valid_0's multi_logloss: 0.336513
[852]	valid_0's multi_logloss: 0.336468
[853]	valid_0's multi_logloss: 0.336462
[854]	valid_0's multi_logloss: 0.336451
[855]	valid_0's multi_logloss: 0.336421
[856]	valid_0's multi_logloss: 0.336401
[857]	valid_0's multi_logloss: 0.33639
[858]	valid_0's multi_logloss: 0.336337
[859]	valid_0's multi_logloss: 0.336371
[860]	valid_0's multi_logloss: 0.336356
[861]	valid_0's multi_logloss: 0.336376
[862]	valid_0's multi_logloss: 0.336361
[863]	valid_0's multi_logloss: 0.336319
[864]	valid_0's multi_logloss: 0.336287
[865]	valid_0's multi_logloss: 0.336278
[

...

[972]	valid_0's multi_logloss: 0.337565
[973]	valid_0's multi_logloss: 0.337545
[974]	valid_0's multi_logloss: 0.337529
[975]	valid_0's multi_logloss: 0.337555
[976]	valid_0's multi_logloss: 0.337563
[977]	valid_0's multi_logloss: 0.337527
[978]	valid_0's multi_logloss: 0.337482
[979]	valid_0's multi_logloss: 0.337486
[980]	valid_0's multi_logloss: 0.337503
[981]	valid_0's multi_logloss: 0.337467
[982]	valid_0's multi_logloss: 0.337433
[983]	valid_0's multi_logloss: 0.337419
[984]	valid_0's multi_logloss: 0.337377
[985]	valid_0's multi_logloss: 0.337363
[986]	valid_0's multi_logloss: 0.337351
[987]	valid_0's multi_logloss: 0.337331
[988]	valid_0's multi_logloss: 0.337319
[989]	valid_0's multi_logloss: 0.337321
[990]	valid_0's multi_logloss: 0.337286
[991]	valid_0's multi_logloss: 0.337293
[992]	valid_0's multi_logloss: 0.337281
[993]	valid_0's multi_logloss: 0.33727
[994]	valid_0's multi_logloss: 0.337246
[995]	valid_0's multi_logloss: 0.337201
[996]	valid_0's multi_logloss: 0.337198
[

[178]	valid_0's multi_logloss: 0.444226
[179]	valid_0's multi_logloss: 0.443296
[180]	valid_0's multi_logloss: 0.44236
[181]	valid_0's multi_logloss: 0.441399
[182]	valid_0's multi_logloss: 0.44051
[183]	valid_0's multi_logloss: 0.439515
[184]	valid_0's multi_logloss: 0.438669
[185]	valid_0's multi_logloss: 0.437779
[186]	valid_0's multi_logloss: 0.437038
[187]	valid_0's multi_logloss: 0.43625
[188]	valid_0's multi_logloss: 0.43563
[189]	valid_0's multi_logloss: 0.43485
[190]	valid_0's multi_logloss: 0.434057
[191]	valid_0's multi_logloss: 0.433384
[192]	valid_0's multi_logloss: 0.432597
[193]	valid_0's multi_logloss: 0.432005
[194]	valid_0's multi_logloss: 0.431371
[195]	valid_0's multi_logloss: 0.430655
[196]	valid_0's multi_logloss: 0.429821
[197]	valid_0's multi_logloss: 0.429155
[198]	valid_0's multi_logloss: 0.428528
[199]	valid_0's multi_logloss: 0.42778
[200]	valid_0's multi_logloss: 0.427138
[201]	valid_0's multi_logloss: 0.426308
[202]	valid_0's multi_logloss: 0.425687
[203]	

[386]	valid_0's multi_logloss: 0.369504
[387]	valid_0's multi_logloss: 0.369405
[388]	valid_0's multi_logloss: 0.369257
[389]	valid_0's multi_logloss: 0.36913
[390]	valid_0's multi_logloss: 0.369011
[391]	valid_0's multi_logloss: 0.368809
[392]	valid_0's multi_logloss: 0.368725
[393]	valid_0's multi_logloss: 0.368672
[394]	valid_0's multi_logloss: 0.368544
[395]	valid_0's multi_logloss: 0.368418
[396]	valid_0's multi_logloss: 0.368327
[397]	valid_0's multi_logloss: 0.36822
[398]	valid_0's multi_logloss: 0.368212
[399]	valid_0's multi_logloss: 0.368084
[400]	valid_0's multi_logloss: 0.367933
[401]	valid_0's multi_logloss: 0.367808
[402]	valid_0's multi_logloss: 0.367708
[403]	valid_0's multi_logloss: 0.367522
[404]	valid_0's multi_logloss: 0.36745
[405]	valid_0's multi_logloss: 0.367373
[406]	valid_0's multi_logloss: 0.367202
[407]	valid_0's multi_logloss: 0.367072
[408]	valid_0's multi_logloss: 0.367003
[409]	valid_0's multi_logloss: 0.366837
[410]	valid_0's multi_logloss: 0.36671
[411

[594]	valid_0's multi_logloss: 0.353018
[595]	valid_0's multi_logloss: 0.352974
[596]	valid_0's multi_logloss: 0.352911
[597]	valid_0's multi_logloss: 0.352884
[598]	valid_0's multi_logloss: 0.352822
[599]	valid_0's multi_logloss: 0.352725
[600]	valid_0's multi_logloss: 0.352679
[601]	valid_0's multi_logloss: 0.35264
[602]	valid_0's multi_logloss: 0.352597
[603]	valid_0's multi_logloss: 0.352577
[604]	valid_0's multi_logloss: 0.352531
[605]	valid_0's multi_logloss: 0.35253
[606]	valid_0's multi_logloss: 0.352483
[607]	valid_0's multi_logloss: 0.352473
[608]	valid_0's multi_logloss: 0.352403
[609]	valid_0's multi_logloss: 0.352356
[610]	valid_0's multi_logloss: 0.352358
[611]	valid_0's multi_logloss: 0.352336
[612]	valid_0's multi_logloss: 0.352281
[613]	valid_0's multi_logloss: 0.352239
[614]	valid_0's multi_logloss: 0.352225
[615]	valid_0's multi_logloss: 0.352146
[616]	valid_0's multi_logloss: 0.35213
[617]	valid_0's multi_logloss: 0.35208
[618]	valid_0's multi_logloss: 0.352031
[619

[808]	valid_0's multi_logloss: 0.34634
[809]	valid_0's multi_logloss: 0.346265
[810]	valid_0's multi_logloss: 0.346243
[811]	valid_0's multi_logloss: 0.346194
[812]	valid_0's multi_logloss: 0.346162
[813]	valid_0's multi_logloss: 0.346179
[814]	valid_0's multi_logloss: 0.346154
[815]	valid_0's multi_logloss: 0.34615
[816]	valid_0's multi_logloss: 0.346131
[817]	valid_0's multi_logloss: 0.346091
[818]	valid_0's multi_logloss: 0.346063
[819]	valid_0's multi_logloss: 0.346047
[820]	valid_0's multi_logloss: 0.346017
[821]	valid_0's multi_logloss: 0.346006
[822]	valid_0's multi_logloss: 0.345935
[823]	valid_0's multi_logloss: 0.345886
[824]	valid_0's multi_logloss: 0.345861
[825]	valid_0's multi_logloss: 0.345858
[826]	valid_0's multi_logloss: 0.345842
[827]	valid_0's multi_logloss: 0.345854
[828]	valid_0's multi_logloss: 0.345896
[829]	valid_0's multi_logloss: 0.34587
[830]	valid_0's multi_logloss: 0.345802
[831]	valid_0's multi_logloss: 0.345778
[832]	valid_0's multi_logloss: 0.345799
[83

In [7]:
# 26번을 제외한 id 리스트
feature = list(train_label[train_label['label'] != 26]['id'])

In [8]:
# train 데이터에서 26번을 삭제시킨다.
temp = []
for n in tqdm(range(train.shape[0])):
    if train['id'][n] in feature:
        temp.append(train.iloc[n])

 84%|████████████████████████████████████████████████████████▎          | 1576326/1875000 [07:07<01:20, 3688.48it/s]


KeyboardInterrupt: 

In [ ]:
# 26번을 삭제시킨 데이터프레임
without = pd.DataFrame(data=np.array(temp), columns=train.columns)
without = without.astype({'id':int, 'time':int})
without

In [ ]:
# https://dacon.io/competitions/official/235689/codeshare/2347?page=1&dtype=recent&ptype=pub
# 데이터 증강을 통해 과접합을 줄여보자 - DACON
# 증강할 데이터 정리

without_train = []

for uid in tqdm(without['id'].unique()):
    temp = np.array(without[without['id'] == uid].iloc[:,2:], np.float32).T
    without_train.append(temp)

without_train = np.array(without_train, np.float32)
without_train = without_train[:,:,:,np.newaxis].reshape(-1,600,6)

without_label = train_label[train_label['label'] != 26]['label']

In [ ]:

# https://dacon.io/competitions/official/235689/codeshare/2347?page=1&dtype=recent&ptype=pub
# 데이터 증강을 통해 과접합을 줄여보자 - DACON
# 원본 데이터 정리
import tensorflow as tf

x_train = []

for uid in tqdm(train['id'].unique()):
    temp = np.array(train[train['id'] == uid].iloc[:,2:], np.float32).T
    x_train.append(temp)

x_train = np.array(x_train, np.float32)
x_train = x_train[:,:,:,np.newaxis].reshape(-1,600,6)

y_train = train_label['label']
y_categorical = tf.keras.utils.to_categorical(y_train)

x_test = []

for uid in tqdm(test['id'].unique()):
    temp = np.array(test[test['id'] == uid].iloc[:,2:], np.float32).T
    x_test.append(temp)

x_test = np.array(x_test, np.float32)
x_test = x_test[:,:,:,np.newaxis].reshape(-1,600,6)

In [ ]:
def aug(data, shift):
    shift_data = np.roll(data, shift, axis=2)
    return shift_data

# 데이터 증강
shift_data = []
shift_label = []
for n in tqdm(range(20)):
    shifted = aug(without_train, n*30)
    shift_data.append(shifted)
    shift_label.append(without_label)

shift_data = np.array(shift_data).reshape(-1,600,6)
shift_label = np.array(shift_label).reshape(-1,1)
shift_categorical = tf.keras.utils.to_categorical(shift_label)

In [ ]:
# 원본 데이터와 증강 데이터 합치기
concat_train = np.concatenate((x_train, shift_data), axis=0)
concat_label = np.concatenate((y_categorical, shift_categorical), axis=0)
print(concat_train.shape)
print(concat_label.shape)

In [ ]:
ml_train = np.array([np.max(concat_train, axis=1),np.min(concat_train, axis=1),np.mean(concat_train, axis=1)]).reshape(-1, 18)
ml_test = np.array([np.max(x_test, axis=1),np.min(x_test, axis=1),np.mean(x_test, axis=1)]).reshape(-1, 18)
ml_label = np.concatenate((np.array(y).reshape(-1,1), shift_label), axis=0)

In [ ]:
ml_label = ml_label.reshape(-1)

In [ ]:
ml_train.shape, ml_test.shape, ml_label.shape, y.shape

In [70]:
model=[]
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=211)
for (train_idx, val_idx) in mskf.split(ml_train, ml_label): 
    print("... Validating on fold {} ...".format(n_fold + 1))
    X_train, X_val = ml_train[train_idx], ml_label[val_idx] 
    y_train, y_val = ml_trainl[train_idx], ml_label[val_idx] 

    lgb_model = lgb.LGBMClassifier(learning_rate=0.05, 
                                                  n_estimators=1000, 
                                                  max_depth=10, 
                                                  min_child_weight=5, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.6,
                                                  objective= 'multiclass', 
                                                  nthread=-1, 
                                                  eval_metric='logloss',
                                                  scale_pos_weight=1, seed=2019)
    
    lgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=50, 
                  eval_metric="logloss")
    
    model.append(lgb_model)

ValueError: Supported target type is: multilabel-indicator. Got 'binary' instead.

In [42]:
ml_train.shape, ml_test.shape

((35265, 18), (782, 18))

In [11]:
# import lightgbm as lgb
import xgboost as xgb
# import catboost as cb

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

In [44]:
# 모델 2번: xgboost

def build_xgboost(split_num, train, target, test, rnd):
    
    params = {
                'colsample_bytree': 0.7,
                'subsample': 0.8,
                'eta': 0.02,
                'max_depth': 12,
                'eval_metric':'mlogloss',
                'objective':'multi:softprob',
                'num_class':61,
                }
    
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((train.shape[0], 61)), np.zeros((test.shape[0], 61))
    
    skf = StratifiedKFold(n_splits=split_num, shuffle=True, random_state=233*rnd)
    for train_idx, val_idx in skf.split(train, target):

        # split train, validation set
        X = train[train_idx]
        y = target[train_idx]
        valid_x = train[val_idx]
        valid_y = target[val_idx]

        d_train = xgb.DMatrix(X, y)
        d_valid = xgb.DMatrix(valid_x, valid_y)
        d_temp = xgb.DMatrix(valid_x)
        d_test = xgb.DMatrix(test)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        
        #run traning
        model = xgb.train(params, d_train, 2000, watchlist, 
                        early_stopping_rounds=50,
                        verbose_eval=100)

        # save feat
        train_pred[val_idx] = model.predict(d_temp)
        test_pred += model.predict(d_test)/split_num
        
        # release
        del model
        gc.collect()
        print('------------------')
        
    return train_pred, test_pred

xgb_train1, xgb_test1 = build_xgboost(5, ml_train, ml_label, ml_test, 1)

ValueError: Found input variables with inconsistent numbers of samples: [35265, 1753072]

In [12]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from catboost import CatBoostClassifier 
import time
k = 10 
models = []
k_fold = KFold(n_splits=k, shuffle=True, random_state=42)
s_kfold = StratifiedKFold(n_splits = k, shuffle = True, random_state = 209) 
mskf = MultilabelStratifiedKFold(n_splits=10, shuffle=True, random_state=211)
for n_fold, (train_idx, val_idx) in enumerate(s_kfold.split(X,y)): 
    X_train, X_val = X[train_idx], X[val_idx] 
    y_train, y_val = y[train_idx], y[val_idx] 
    l_train, l_val = labels[train_idx], labels[val_idx]  
        
    ##### augment data #####
    print("... Augmenting Data ...")
    X_augmented = [] 
    y_augmented = []  
    l_augmented = [] 
    idx = 3125
    for i in tqdm(range(X_train.shape[0]), position = 0, leave = True): 
        for j in range(15): 
            shifted = np.roll(X_train[i], int(random.random() * 600)) 
            X_augmented.append(shifted) 
            y_augmented.append(y_train[i])     
            l_augmented.append(idx) 
            idx += 1 
    X_augmented = np.asarray(X_augmented) 
    y_augmented = np.asarray(y_augmented)
    l_augmented = np.asarray(l_augmented) 
    X_train = np.concatenate([X_train, X_augmented]) 
    y_train = np.concatenate([y_train, y_augmented])
    l_train = np.concatenate([l_train, l_augmented])  
    
    '''
    ##### feature engineering data ##### 
    print("... Feature Engineering ...")
    X_fourier_real = [] 
    X_fourier_imag = [] 
    for i in tqdm(range(X_train.shape[0]), position = 0, leave = True):  
        real_part = np.fft.fft(X_train[i]).real 
        imag_part = np.fft.fft(X_train[i]).imag 
        X_fourier_real.append(real_part)
        X_fourier_imag.append(imag_part) 
    
    X_fourier_real = np.asarray(X_fourier_real)  
    X_fourier_imag = np.asarray(X_fourier_imag)
    X_train = np.concatenate([X_train, X_fourier_real, X_fourier_imag], axis = 2)   
    
    
    X_val_fourier_real = [] 
    X_val_fourier_imag = [] 
    for i in tqdm(range(X_val.shape[0]), position = 0, leave = True):
        real_part = np.fft.fft(X_val[i]).real 
        imag_part = np.fft.fft(X_val[i]).imag 
        X_val_fourier_real.append(real_part) 
        X_val_fourier_imag.append(imag_part)
    
    X_val_fourier_real = np.asarray(X_val_fourier_real) 
    X_val_fourier_imag = np.asarray(X_val_fourier_imag)
    X_val = np.concatenate([X_val, X_val_fourier_real, X_val_fourier_imag], axis = 2)  
    '''
    
    
    ##### create dataframe for X_train and X_val ##### 
    print("... Creating Dataframe ...")
    l_train_flat = [] 
    for i in tqdm(range(len(l_train))):
        for j in range(600): 
            l_train_flat.append(l_train[i]) 
    l_train_flat = np.asarray(l_train_flat)
    
    l_val_flat = [] 
    for i in tqdm(range(len(l_val))): 
        for j in range(600): 
            l_val_flat.append(l_val[i]) 
    l_val_flat = np.asarray(l_val_flat)
    
    start = time.time() 
    print("... Aggregating Train DataFrame ...")
    train_df = pd.DataFrame(np.concatenate(X_train)) 
    print(train_df.shape)
    train_df.insert(0,'id',l_train_flat) 
    #features = ['id', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
    features = ['id',0,1,2,3,4,5]
    train_df_processed = train_df[features].groupby('id').agg(['max', 'min', 'mean', 'var', 'std']) 
    
    print("... Aggregating Val DataFrame ...") 
    val_df = pd.DataFrame(np.concatenate(X_val)) 
    val_df.insert(0, 'id', l_val_flat)  
    val_df_processed = val_df[features].groupby('id').agg(['max', 'min', 'mean', 'var', 'std'])   
    
    print("... Colum preprocess ...")
    colum_list = []
    print(train_df_processed.shape[1])
    for i in tqdm(range(int(train_df_processed.shape[1]))):
        colum_list.append(i)
    train_df_processed.columns = colum_list
    val_df_processed.columns = colum_list
    
    print("total pre-processing time = {}".format(time.time() - start))  
    
    
    print("... Begin Training ...") 
    print("Validating on fold {}".format(n_fold))
    
#     lgb_model = lgb.LGBMClassifier(learning_rate=0.02, 
#                                                   n_estimators=2500, 
#                                                   max_depth=10, 
#                                                   min_child_weight=5, 
#                                                   gamma=0, 
#                                                   subsample=0.8, 
#                                                   colsample_bytree=0.8,
#                                                   objective= 'multiclass', 
#                                                   nthread=-1, 
#                                                   eval_metric='logloss',
#                                                   scale_pos_weight=1, seed=2019)
    
#     lgb_model.fit(train_df_processed, y_train, eval_set = [(val_df_processed, y_val)], early_stopping_rounds=50, 
#                   eval_metric="logloss")

    xgb_model = xgb.XGBClassifier(learning_rate=0.01, 
                                                  n_estimators=3000, 
                                                  max_depth=10, 
                                                  min_child_weight=5, 
                                                  gamma=0, 
                                                  subsample=0.9, 
                                                  colsample_bytree=0.9,
                                                  objective= 'multi:softmax', 
                                                  nthread=-1, 
                                                  eval_metric='mlogloss',
                                                  scale_pos_weight=1, seed=2019)
    xgb_model.fit(train_df_processed, y_train, eval_set = [(val_df_processed, y_val)], early_stopping_rounds=50, 
                   eval_metric="mlogloss")
#     model = CatBoostClassifier(iterations = 20000, 
#                                learning_rate = 0.01, 
#                                depth = 8, 
#                                loss_function = 'MultiClass',
#                                eval_metric = 'MultiClass',
#                                use_best_model = True, 
#                                verbose = 200,
#                                random_seed = 88888,
#                                classes_count = 61) 


#     model.fit(train_df_processed, y_train, eval_set = (val_df_processed, y_val))
    models.append(xgb_model)

  7%|█████▏                                                                    | 198/2812 [00:00<00:01, 1965.62it/s]

... Augmenting Data ...


  2%|█▎                                                                       | 803/44992 [00:00<00:05, 7971.81it/s]

... Creating Dataframe ...


100%|███████████████████████████████████████████████████████████████████████████| 313/313 [00:00<00:00, 8052.46it/s]


...

[459]	validation_0-mlogloss:1.09780
[460]	validation_0-mlogloss:1.09715
[461]	validation_0-mlogloss:1.09671
[462]	validation_0-mlogloss:1.09617
[463]	validation_0-mlogloss:1.09550
[464]	validation_0-mlogloss:1.09499
[465]	validation_0-mlogloss:1.09452
[466]	validation_0-mlogloss:1.09390
[467]	validation_0-mlogloss:1.09339
[468]	validation_0-mlogloss:1.09284
[469]	validation_0-mlogloss:1.09240
[470]	validation_0-mlogloss:1.09191
[471]	validation_0-mlogloss:1.09150
[472]	validation_0-mlogloss:1.09095
[473]	validation_0-mlogloss:1.09046
[474]	validation_0-mlogloss:1.08990
[475]	validation_0-mlogloss:1.08939
[476]	validation_0-mlogloss:1.08891
[477]	validation_0-mlogloss:1.08845
[478]	validation_0-mlogloss:1.08791
[479]	validation_0-mlogloss:1.08722
[480]	validation_0-mlogloss:1.08672
[481]	validation_0-mlogloss:1.08612
[482]	validation_0-mlogloss:1.08575
[483]	validation_0-mlogloss:1.08521
[484]	validation_0-mlogloss:1.08479
[485]	validation_0-mlogloss:1.08424
[486]	validation_0-mlogloss:

[687]	validation_0-mlogloss:1.02252
[688]	validation_0-mlogloss:1.02245
[689]	validation_0-mlogloss:1.02222
[690]	validation_0-mlogloss:1.02192
[691]	validation_0-mlogloss:1.02153
[692]	validation_0-mlogloss:1.02132
[693]	validation_0-mlogloss:1.02108
[694]	validation_0-mlogloss:1.02094
[695]	validation_0-mlogloss:1.02067
[696]	validation_0-mlogloss:1.02044
[697]	validation_0-mlogloss:1.02011
[698]	validation_0-mlogloss:1.01992
[699]	validation_0-mlogloss:1.01974
[700]	validation_0-mlogloss:1.01949
[701]	validation_0-mlogloss:1.01919
[702]	validation_0-mlogloss:1.01914
[703]	validation_0-mlogloss:1.01882
[704]	validation_0-mlogloss:1.01860
[705]	validation_0-mlogloss:1.01838
[706]	validation_0-mlogloss:1.01825
[707]	validation_0-mlogloss:1.01817
[708]	validation_0-mlogloss:1.01794
[709]	validation_0-mlogloss:1.01775
[710]	validation_0-mlogloss:1.01780
[711]	validation_0-mlogloss:1.01757
[712]	validation_0-mlogloss:1.01753
[713]	validation_0-mlogloss:1.01732
[714]	validation_0-mlogloss:

[915]	validation_0-mlogloss:0.99481
[916]	validation_0-mlogloss:0.99483
[917]	validation_0-mlogloss:0.99470
[918]	validation_0-mlogloss:0.99465
[919]	validation_0-mlogloss:0.99455
[920]	validation_0-mlogloss:0.99457
[921]	validation_0-mlogloss:0.99448
[922]	validation_0-mlogloss:0.99452
[923]	validation_0-mlogloss:0.99452
[924]	validation_0-mlogloss:0.99439
[925]	validation_0-mlogloss:0.99447
[926]	validation_0-mlogloss:0.99438
[927]	validation_0-mlogloss:0.99426
[928]	validation_0-mlogloss:0.99415
[929]	validation_0-mlogloss:0.99412
[930]	validation_0-mlogloss:0.99394
[931]	validation_0-mlogloss:0.99412
[932]	validation_0-mlogloss:0.99397
[933]	validation_0-mlogloss:0.99395
[934]	validation_0-mlogloss:0.99383
[935]	validation_0-mlogloss:0.99378
[936]	validation_0-mlogloss:0.99376
[937]	validation_0-mlogloss:0.99365
[938]	validation_0-mlogloss:0.99366
[939]	validation_0-mlogloss:0.99363
[940]	validation_0-mlogloss:0.99361
[941]	validation_0-mlogloss:0.99350
[942]	validation_0-mlogloss:

[1139]	validation_0-mlogloss:0.98578
[1140]	validation_0-mlogloss:0.98585
[1141]	validation_0-mlogloss:0.98575
[1142]	validation_0-mlogloss:0.98567
[1143]	validation_0-mlogloss:0.98561
[1144]	validation_0-mlogloss:0.98560
[1145]	validation_0-mlogloss:0.98560
[1146]	validation_0-mlogloss:0.98569
[1147]	validation_0-mlogloss:0.98560
[1148]	validation_0-mlogloss:0.98544
[1149]	validation_0-mlogloss:0.98545
[1150]	validation_0-mlogloss:0.98552
[1151]	validation_0-mlogloss:0.98540
[1152]	validation_0-mlogloss:0.98541
[1153]	validation_0-mlogloss:0.98532
[1154]	validation_0-mlogloss:0.98527
[1155]	validation_0-mlogloss:0.98519
[1156]	validation_0-mlogloss:0.98516
[1157]	validation_0-mlogloss:0.98505
[1158]	validation_0-mlogloss:0.98509
[1159]	validation_0-mlogloss:0.98509
[1160]	validation_0-mlogloss:0.98503
[1161]	validation_0-mlogloss:0.98503
[1162]	validation_0-mlogloss:0.98504
[1163]	validation_0-mlogloss:0.98500
[1164]	validation_0-mlogloss:0.98495
[1165]	validation_0-mlogloss:0.98489
[

[1361]	validation_0-mlogloss:0.98177
[1362]	validation_0-mlogloss:0.98171
[1363]	validation_0-mlogloss:0.98157
[1364]	validation_0-mlogloss:0.98152
[1365]	validation_0-mlogloss:0.98152
[1366]	validation_0-mlogloss:0.98150
[1367]	validation_0-mlogloss:0.98145
[1368]	validation_0-mlogloss:0.98141
[1369]	validation_0-mlogloss:0.98143
[1370]	validation_0-mlogloss:0.98137
[1371]	validation_0-mlogloss:0.98128
[1372]	validation_0-mlogloss:0.98138
[1373]	validation_0-mlogloss:0.98146
[1374]	validation_0-mlogloss:0.98141
[1375]	validation_0-mlogloss:0.98136
[1376]	validation_0-mlogloss:0.98139
[1377]	validation_0-mlogloss:0.98129
[1378]	validation_0-mlogloss:0.98119
[1379]	validation_0-mlogloss:0.98109
[1380]	validation_0-mlogloss:0.98104
[1381]	validation_0-mlogloss:0.98103
[1382]	validation_0-mlogloss:0.98098
[1383]	validation_0-mlogloss:0.98097
[1384]	validation_0-mlogloss:0.98091
[1385]	validation_0-mlogloss:0.98088
[1386]	validation_0-mlogloss:0.98077
[1387]	validation_0-mlogloss:0.98085
[

In [22]:
X = tf.reshape(np.array(concat_train.iloc[:,2:]), [-1,600,6]) 
X = np.asarray(X) 
y = train_label['label'].values 
y = tf.keras.utils.to_categorical(y) 
X.shape, y.shape

((35265, 600, 6), (35265, 61, 2))

In [9]:
# from tensorflow.keras.layers import Dense, Lambda, dot, Activation, concatenate
# from tensorflow.keras.layers import Layer


# class Attention(Layer):

#     def __init__(self, **kwargs):
#         super().__init__(**kwargs)

#     def __call__(self, hidden_states):
#         """
#         Many-to-one attention mechanism for Keras.
#         @param hidden_states: 3D tensor with shape (batch_size, time_steps, input_dim).
#         @return: 2D tensor with shape (batch_size, 128)
#         @author: felixhao28.
#         """
#         hidden_size = int(hidden_states.shape[2])
#         print(hidden_size)
#         # Inside dense layer
#         #              hidden_states            dot               W            =>           score_first_part
#         # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
#         # W is the trainable weight matrix of attention Luong's multiplicative style score
#         score_first_part = Dense(hidden_size, use_bias=False, name='attention_score_vec')(hidden_states)
#         #            score_first_part           dot        last_hidden_state     => attention_weights
#         # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
#         h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(hidden_states)
#         score = dot([score_first_part, h_t], [2, 1], name='attention_score')
#         attention_weights = Activation('softmax', name='attention_weight')(score)
#         # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
#         context_vector = dot([hidden_states, attention_weights], [1, 1], name='context_vector')
#         pre_activation = concatenate([context_vector, h_t], name='attention_output')
#         attention_vector = Dense(128, use_bias=False, activation='tanh', name='attention_vector')(pre_activation)
#         return attention_vector

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.layers import Input, TimeDistributed, GRU, Conv2D, Conv1D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, MaxPooling1D, BatchNormalization, concatenate, ConvLSTM2D, Reshape, Conv3D, Flatten, LSTM, Dense, Dropout, Add
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D, GlobalMaxPool1D, Attention
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import Sequential, load_model, save_model
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import keras
from keras_self_attention import SeqSelfAttention
from attention import Attention

def build_model():  
    model = Sequential()
    model.add(LSTM(128, input_shape = (600,6), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(61, activation='softmax'))
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'],
    )
    return model



In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import time
import gc
k = 10 
models = []
k_fold = KFold(n_splits=k, shuffle=True, random_state=42)
s_kfold = StratifiedKFold(n_splits = k, shuffle = True, random_state = 209) 
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=211)
for n_fold, (train_idx, val_idx) in enumerate(mskf.split(concat_train, concat_label)): 
    print("... Validating on fold {} ...".format(n_fold + 1))
    X_train, X_val = concat_train[train_idx], concat_train[val_idx] 
    y_train, y_val = concat_label[train_idx], concat_label[val_idx] 
        
    print(X_train.shape, y_train.shape)
    ##### train model #####  
    print("... Training ...") 
    model = build_model()
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']) 
    model_path = './kfold/' + str(n_fold+1) + 'epoch_{epoch:03d}_val_{val_loss:.3f}.h5'
    #learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss', patience = 2, verbose = 1, factor = 0.8)
    checkpoint = ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 7) 

    model.fit(X_train,
              y_train,
              epochs = 200,
              batch_size = 32,
              validation_data = (X_val, y_val),
              callbacks = [checkpoint, early_stopping])
    
    # release
    del model
    gc.collect()
    print('------------------')

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
import time
k = 10 
models = []
k_fold = KFold(n_splits=k, shuffle=True, random_state=42)
s_kfold = StratifiedKFold(n_splits = k, shuffle = True, random_state = 209) 
for n_fold, (train_idx, val_idx) in enumerate(k_fold.split(X)): 
    print("... Validating on fold {} ...".format(n_fold + 1))
    X_train, X_val = X[train_idx], X[val_idx] 
    y_train, y_val = y[train_idx], y[val_idx] 
        
    ##### augment data #####
    print("... Augmenting Data ...")
    X_augmented = [] 
    y_augmented = []  
    for i in tqdm(range(X_train.shape[0])): 
        for j in range(30): 
            shifted = np.roll(X_train[i], int(random.random() * 600)) 
            X_augmented.append(shifted) 
            y_augmented.append(y_train[i])     
    X_augmented = np.asarray(X_augmented) 
    y_augmented = np.asarray(y_augmented)
    X_train = np.concatenate([X_train, X_augmented]) 
    y_train = np.concatenate([y_train, y_augmented])
    
#     scaler = StandardScaler()
#     X_train = scaler.fit_transform(X_train.reshape(1, -1))
#     X_val = scaler.transform(X_val.reshape(1, -1))
    print(X_train.shape, y_train.shape)
    ##### train model #####  
    print("... Training ...") 
    model = build_model()
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']) 
    model_path = './kfold/' + str(n_fold+1) + 'epoch_{epoch:03d}_val_{val_loss:.3f}.h5'
    #learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss', patience = 2, verbose = 1, factor = 0.8)
    checkpoint = ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 7) 

    model.fit(X_train,
              y_train,
              epochs = 200,
              batch_size = 32,
              validation_data = (X_val, y_val),
              callbacks = [checkpoint, early_stopping])

In [9]:
features = ['id', 'acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z']
X_test = test[features].groupby('id').agg(['max', 'min', 'mean', 'var', 'std'])

In [10]:
X_test.shape

(782, 30)

In [31]:
x_test.shape

(782, 600, 6)

In [35]:
gru1 = load_model('./kfold/1epoch_019_val_0.031.h5')
gru2 = load_model('./kfold/2epoch_029_val_0.029.h5') 
gru3 = load_model('./kfold/3epoch_033_val_0.018.h5')  
gru4 = load_model('./kfold/4epoch_027_val_0.025.h5') 
gru5 = load_model('./kfold/5epoch_024_val_0.014.h5')

ValueError: A `Dot` layer should be called on a list of 2 inputs.

In [11]:
# 예측
pred_y_list = []
for model in models:
    pred_y = model.predict_proba(X_test)
    pred_y_list.append(pred_y.reshape(-1,61))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [12]:
submission.iloc[:,1:] = pred_ensemble
submission

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,0.000232,0.000326,0.000443,0.000273,0.000716,0.000143,0.000449,0.000720,0.000356,...,0.002450,0.000511,0.000474,0.000555,0.000283,0.000239,0.000444,0.029680,0.000122,0.005492
1,3126,0.000270,0.000200,0.000080,0.000292,0.000105,0.000115,0.000035,0.000518,0.000156,...,0.000112,0.000022,0.000045,0.000253,0.000132,0.000354,0.000108,0.000038,0.000039,0.000236
2,3127,0.000921,0.507997,0.000684,0.003619,0.000333,0.000090,0.083425,0.001745,0.000715,...,0.000133,0.000147,0.000074,0.001543,0.000323,0.001039,0.000103,0.001026,0.000153,0.057535
3,3128,0.001091,0.000342,0.000354,0.000166,0.000375,0.000120,0.000070,0.000874,0.000198,...,0.000300,0.000046,0.000605,0.000177,0.000244,0.000139,0.000500,0.000216,0.000050,0.004622
4,3129,0.001172,0.000265,0.000169,0.001557,0.000195,0.000297,0.000089,0.000333,0.000801,...,0.000170,0.000038,0.000170,0.000102,0.000165,0.000506,0.000339,0.000077,0.000198,0.000187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0.012212,0.000707,0.000904,0.002834,0.057677,0.001226,0.000304,0.000930,0.002184,...,0.004714,0.000150,0.000945,0.000277,0.000677,0.001509,0.000841,0.000282,0.000665,0.000583
778,3903,0.000594,0.000066,0.000044,0.000133,0.000153,0.000238,0.000035,0.000196,0.000247,...,0.000094,0.000017,0.000030,0.000035,0.000068,0.000546,0.000041,0.000020,0.000437,0.000097
779,3904,0.000480,0.000325,0.000122,0.000804,0.000139,0.000108,0.000064,0.000339,0.000433,...,0.000111,0.000030,0.000158,0.000157,0.000153,0.000111,0.000185,0.000087,0.000039,0.001699
780,3905,0.000072,0.000452,0.000036,0.000057,0.000014,0.000011,0.003944,0.000301,0.000069,...,0.000056,0.000006,0.000012,0.000243,0.000042,0.000037,0.000042,0.000048,0.000013,0.000110


In [13]:
submission.to_csv('lgb_aug_k.csv', index=False)